## Zefys Splitter

Aufteilen und Speichern nach Zeitungsartikeln

In [2]:
import re
import csv
import os
import pandas as pd
from bs4 import BeautifulSoup
from datetime import date



In [6]:
abs_filepath = r'C:\Users\Martin\sciebo\Documents\MeineJupyterNotebooks\zefys-scraper-version-2'

# Zeitung auswählen
abbr_newspaper = 'NM' # 'PC'
# Artikel Counter auf den richtigen Wert setzen, wenn Datensatz Neueste Mittheilungen ergänzt wird  || Refactoring! verbessern!

xml_data_folder = f'\data_{abbr_newspaper}_xml'
meta_data_folder = '\meta_data'
txt_data_folder = f'data_{abbr_newspaper}_txt_V3'

meta_data_file = f'200524-metadaten-amtspresse-article-text.csv'

path = os.path.join(abs_filepath, txt_data_folder)
try:
    os.mkdir(path)
except:
    print('Ordner existiert bereits.')

Ordner existiert bereits.


In [7]:
article_counter = 8718

if meta_data_file not in os.listdir(abs_filepath +  meta_data_folder):

# Anlegen der csv-Datei mit den Metadaten
    with open(abs_filepath + meta_data_folder + '\\' + meta_data_file, 'w', 
             encoding="utf-8", 
             newline='') as csvfile:
        writer = csv.writer(csvfile, 
                           delimiter=',', 
                           quotechar='"', 
                           quoting=csv.QUOTE_NONNUMERIC)
        writer.writerow(['filename', 'full_date', 'year', 'year_month', 'article_count', \
                        'article_counts_per_issue','article_length_chars', \
                        'article_length_words', 'newspaper', 'headline', 'article_text'])

# Iterieren über die Dateien im xml-Dateien-Ordner:
for root, dirs, files in os.walk(abs_filepath + xml_data_folder): # || Refactoring! Nachgucken! Verbessern!
    pass # print(file)
    
for xml_file in files:
    with open(abs_filepath + xml_data_folder + '\\' + xml_file, 'r', encoding='utf8') as f:
        text = f.read()

    text_header, text_body = text.split('</datum>') # abtrennen des Headers
    try:
        text_articles, text_footer = text_body.split('<fusszeile>') # abtrennen des Footers
        article_list = text_articles.split('<hr/>') # aufteilen der Zeitung bei <hr/>
    except:
        article_list = text_body.split('<hr/>')

    for article in article_list:   
     
        if article.count('<untertitel>') >= 2: # mit <untertitel> splitten
            article_splits = article.split('<untertitel>')
            article_splits.remove(article_splits[0]) #  xml vor 1. <untertitel> löschen
            article_list.extend(article_splits)       

    counter = 1

    for article in article_list:
        
        # Überschriften mit regex herausfiltern || Refactoring? Alternative Beautiful Soup?
        pattern = re.compile('<fett[^>]*>(.+?)</fett>')
        heading = re.search(pattern, article)         
        
        # print(heading)
        if heading != None:
            headline = heading[0]
            headline = re.sub('<[^>]*>', "", headline)
            
        if 'Wochenschau' not in headline and heading == None:
            headline = "Ohne Überschrift"
    
        # print(article_counter, headline)
        
        # xml-tag entfernen, um weitere Schritte durchzuführen
        article = article.replace('</titelkopf>','')

        # xml-tags entfernen
        soup = BeautifulSoup(article) 
        article_without_xml = soup.get_text()     

        # entfernen von nicht gewolltem Mark-Up-Text
        article_without_xml = re.sub( 
            r'sachkommentar\d+\*\d-\d\d-\d', 
            '', 
            article_without_xml)
        
        # Datumsformat aus xml-Dateiname-konvertieren
        date_format = pd.to_datetime(xml_file[:-4]) 
        
        # Zuordnen der Variablen für Metadaten-Datei
        filename = xml_file[:-4] + '_' + str(counter).zfill(2) + '.txt'              
        full_date = xml_file[:-4]
        year = date_format.strftime('%Y')
        year_month = date_format.strftime('%Y-%m')
        article_count = article_counter
        article_counts_per_issue = len(article_list)
        article_length_chars = len(article)
        article_length_words = len(article.split(' '))
        headline = headline
        newspaper = abbr_newspaper
        article_text = article_without_xml.strip()           
        

        # Einstellen der Artikel Länge, die nicht beachtet werden soll
        if article_length_words < 5:  
            continue # ersetzen mit continue     

        # Speichern der Metadaten in einer Zeile der csv-Datei
        
        with open(abs_filepath + meta_data_folder + '\\' + meta_data_file, 'a', 
                 encoding="utf-8", 
                 newline='') as csvfile:
            writer = csv.writer(csvfile, 
                               delimiter=',', 
                               quotechar='"', 
                               quoting=csv.QUOTE_NONNUMERIC)
            writer.writerow([filename, full_date, year, year_month, article_count, \
                            article_counts_per_issue, article_length_chars, \
                            article_length_words, newspaper, headline, article_text])

        # Speichern der einzelnen Artikel in txt-Dateien
#         with open(path + '\\' + filename, 'w', encoding='utf8') as f:
#             f.write(article_text)

        # Counter hochzählen
        counter +=1
        article_counter +=1
print(article_counter)

29143


In [ ]:
# to do

# corpus mit NLTK weiterverarbeiten!!!

# Benennung des Artikel mit Nummer hinter dem Datum, am besten mit .zfill(2), counter kann ganz gut genutzt werden!
# article speichern in txt-format
# weitere Feature-Werte herausziehen (?Anzahl Wörter?, Anzahl Buchstaben), Dateiname, Anzahl der Artikel pro Ausgabe (später als kategorische Spalte nutzbar!), 

# Refactoring: For-Schleife mit ausgelagerten Funktion übersichtlicher machen

# Ungleicheit von Anzahl der gedruckten Zeilen und len(article_list) eruieren | jeweils der erste Eintrag in der Liste scheint leer zu sein || Lösung: article = article.replace('</titelkopf','')

# sachkommentar und weitere Angaben mit RegEx rauslöschen mit replace(REGEX, ' ')
  # RegEx, weil die Zahl des Sachkommentars durchnummeriert ist! || # r'sachkommentar\d+\*\d-\d\d-\d'